<a href="https://colab.research.google.com/github/Abishethvarman/KD-Sentiment-Analysis/blob/main/DSAI2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 18APC3538_Research_Proposal.gdoc
'19APC3970_DBMS_Class Assignment 1.zip'
 19APC3970-English.pdf
 19APC3970_FWT_QUIZ02.pdf
'19APC3970 HTML-01 (1).zip'
'19APC3970 HTML-01.zip'
'19APC3970 HTML _1.rar'
 19APC3970.LOG2.docx
 19APC3970.LOG.docx
 19APC3970_QUIZ02.pdf
 19APC3970-Quiz03.pdf
 20240216_091107.jpg
 20240216_091245.jpg
 20240216_091320.jpg
 20240216_095746.jpg
 20240216_170935.jpg
 Abstract
 Abstract.gdoc
'ADScAI_142_Payment (1).pdf'
 ADScAI_142_Payment.pdf
 Artwork-Gobihanath
 Attendence1.jpg
'Author Declaration Form IRCFoT-2025_V2.docx'
'Balancing Academic and Non-Academic Workload: Stress Assessment .gform'
'Balancing Academic and Non-Academic Workload: Stress Assessment  (Responses).gsheet'
 Banana1.jpg
 Banana2.jpg
 BI-Report.gdoc
'CamScanner 08-26-2022 13.34.jpg'
'CamScanner 11-05-2021 11.58_1.pdf'
'CamScanner 11-05-2021 11.58.pdf'
'Capstone Project - Group 22'
 CIS
 Classroom
'Colab Notebooks'
'Community Doc.gdoc'
'data - final.xlsx'
 Dataset
 Dia

In [2]:
# Imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
%cd /content/gdrive/MyDrive/Dataset

/content/gdrive/MyDrive/Dataset


In [16]:
df = pd.read_csv("movie.csv")

In [17]:
df=df.head(20000)

In [18]:
df

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
19995,"I was required to watch the movie for my work,...",0
19996,"""White Noise"" had potential to be one of the m...",0
19997,The Five Deadly Venoms is a great kung-fu acti...,1
19998,Ali G Indahouse has got to be one of the funni...,1


In [19]:
label_counts = df['label'].value_counts()
label_counts

,count
label,
0,10024
1,9976


In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Step 1: Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df["text"])
y = df["label"]

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train Naive Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Step 4: Predict
y_pred = nb_model.predict(X_test)

# Step 5: Evaluate
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

# Step 6: Print results
print("Naive Bayes Classifier Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)


Naive Bayes Classifier Performance:
Accuracy : 0.8438
Precision: 0.8485
Recall   : 0.8414
F1 Score : 0.8450
Confusion Matrix:
[[1672  304]
 [ 321 1703]]


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Step 1: Prepare text and labels
texts = df["text"].astype(str).tolist()
labels = df["label"].tolist()

# Step 2: Tokenize text
vocab_size = 10000
max_len = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Step 4: Define LSTM model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Step 5: Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=5, batch_size=32, validation_split=0.1, verbose=2)

# Step 6: Evaluate
y_pred_prob = model.predict(np.array(X_test))
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

# Step 7: Print metrics
print("\nLSTM Model Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


450/450 - 36s - 81ms/step - accuracy: 0.6378 - loss: 0.6212 - val_accuracy: 0.7244 - val_loss: 0.5691
Epoch 2/5
450/450 - 42s - 93ms/step - accuracy: 0.8251 - loss: 0.4202 - val_accuracy: 0.7500 - val_loss: 0.5056
Epoch 3/5
450/450 - 36s - 79ms/step - accuracy: 0.8856 - loss: 0.2970 - val_accuracy: 0.7962 - val_loss: 0.4723
Epoch 4/5
450/450 - 43s - 95ms/step - accuracy: 0.9284 - loss: 0.2049 - val_accuracy: 0.7956 - val_loss: 0.5142
Epoch 5/5
450/450 - 39s - 86ms/step - accuracy: 0.9473 - loss: 0.1476 - val_accuracy: 0.7769 - val_loss: 0.6148
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step

LSTM Model Performance:
Accuracy : 0.7780
Precision: 0.7883
Recall   : 0.7673
F1 Score : 0.7777
Confusion Matrix:
[[1559  417]
 [ 471 1553]]


In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Step 1: Extract features and labels
texts = df['text'].astype(str).values
labels = df['label'].values

# Step 2: Convert text to TF-IDF vectors
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X = vectorizer.fit_transform(texts)

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Step 4: Train SVM model
svm_model = LinearSVC()
svm_model.fit(X_train, y_train)

# Step 5: Predictions and evaluation
y_pred = svm_model.predict(X_test)

print("SVM Evaluation Metrics:")
print(f"Accuracy : {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred, zero_division=0):.4f}")
print(f"Recall   : {recall_score(y_test, y_pred, zero_division=0):.4f}")
print(f"F1 Score : {f1_score(y_test, y_pred, zero_division=0):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


SVM Evaluation Metrics:
Accuracy : 0.8635
Precision: 0.8630
Recall   : 0.8681
F1 Score : 0.8655

Confusion Matrix:
[[1697  279]
 [ 267 1757]]


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Parameters
max_vocab_size = 5000
max_seq_len = 100
embedding_dim = 100
batch_size = 32
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'].astype(str).values)
sequences = tokenizer.texts_to_sequences(df['text'].astype(str).values)
padded_sequences = pad_sequences(sequences, maxlen=max_seq_len, padding='post')

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['label'].values)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# PyTorch Dataset
class SentimentDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = SentimentDataset(X_train, y_train)
test_dataset = SentimentDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# CNN Model
class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes=1):
        super(CNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=5)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(128 * ((max_seq_len - 5 + 1) // 2), 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)  # (batch, embed_dim, seq_len)
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(torch.relu(self.fc1(x)))
        return self.sigmoid(self.fc2(x)).squeeze()

model = CNNClassifier(vocab_size=max_vocab_size, embed_dim=embedding_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = (outputs > 0.5).int().cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(y_batch.numpy())

# Metrics
print("\nCNN Evaluation Metrics:")
print(f"Accuracy : {accuracy_score(y_true, y_pred):.4f}")
print(f"Precision: {precision_score(y_true, y_pred, zero_division=0):.4f}")
print(f"Recall   : {recall_score(y_true, y_pred, zero_division=0):.4f}")
print(f"F1 Score : {f1_score(y_true, y_pred, zero_division=0):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

Epoch 1: 100%|██████████| 500/500 [00:27<00:00, 17.92it/s]


Epoch 1 Loss: 0.6888


Epoch 2: 100%|██████████| 500/500 [00:25<00:00, 19.35it/s]


Epoch 2 Loss: 0.5810


Epoch 3: 100%|██████████| 500/500 [00:26<00:00, 18.91it/s]


Epoch 3 Loss: 0.4467


Epoch 4: 100%|██████████| 500/500 [00:26<00:00, 19.02it/s]


Epoch 4 Loss: 0.3731


Epoch 5: 100%|██████████| 500/500 [00:26<00:00, 18.64it/s]


Epoch 5 Loss: 0.3158

CNN Evaluation Metrics:
Accuracy : 0.8260
Precision: 0.7959
Recall   : 0.8824
F1 Score : 0.8369
Confusion Matrix:
 [[1518  458]
 [ 238 1786]]


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 1. Load and prepare data
X = df['text']
y = df['label']

# 2. Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# 4. Train KNN classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# 5. Predict and evaluate
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
conf_matrix = confusion_matrix(y_test, y_pred)

# 6. Display results
print("KNN Classifier Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)


KNN Classifier Metrics:
Accuracy : 0.6957
Precision: 0.6802
Recall   : 0.7525
F1 Score : 0.7145

Confusion Matrix:
[[1260  716]
 [ 501 1523]]
